# 验证和导入用户项目交互数据<a class="anchor" id="top"></a>

在本笔记本中，您将选择一个数据集，以准备用于 Amazon Personalize。

1. [简介](#intro)
1. [选择数据集或数据源](#source)
1. [准备数据](#prepare)
1. [创建数据集组和交互数据集](#group_dataset)
1. [配置 S3 存储桶和 IAM 角色](#bucket_role)
1. [导入交互数据](#import)

## 简介<a class="anchor" id="intro"></a>

在大部分情况下，Amazon Personalize 中的算法（称为配方）在很大程度上寻求解决不同的任务，如下所述：

1. **用户个性化** – 支持 ALL HRNN 工作流程/满足用户个性化需求的新版本，我们将在此处使用该版本。
1. **HRNN 和 HRNN-Metadata** – 根据以前用户与项的交互推荐项目。
1. **HRNN-Coldstart** – 推荐尚未提供交互数据的新项目。
1. **Personalized-Ranking** – 获取一个项目集合，然后使用类似 HRNN 的方法按照可能感兴趣的顺序对它们进行排序。
1. **SIMS（类似项目）**– 根据给定的项目，推荐用户也可交互的其他项目。
1. **Popularity-Count** – 如果 HRNN 或 HRNN-Metadata 没有答案，则推荐最受欢迎的项目 – 这是默认返回值。

无论使用案例如何，所有算法都共享基于用户-项目-交互数据的学习基础，该数据由 3 个核心属性定义：

1. **UserID** – 参与交互的用户
1. **ItemID** – 用户与之交互的项目
1. **Timestamp** – 交互发生的时间

我们还支持按以下值定义的事件类型和事件值：

1. **事件类型** – 事件的分类标签（浏览、已购买、已评价等）。
1. **事件值** – 与发生的事件类型对应的值。一般来说，我们在事件类型中寻找 0 和 1 之间的归一化值。例如，如果完成交易有三个阶段（点击、添加到购物车和购买），则每个阶段的事件值分别为 0.33、0.66 和 1.0。

事件类型和事件值字段是附加数据，可用于筛选为训练个性化模型而发送的数据。在这个特定的练习中，我们没有事件类型或事件值。

## 选择数据集或数据源<a class="anchor" id="source"></a>
[返回页首](#top)

正如我们所提到的，用户-项目-交互数据是开始使用服务的关键。这意味着我们需要寻找生成这种数据的使用案例，几个常见示例包括：

1. 视频点播应用
1. 电子商务平台
1. 社交媒体聚合器/平台

使用一些指导原则可帮助确定适合 Personalize 的问题的范围。我们建议将下面的值作为起点，不过[官方限制](https://docs.aws.amazon.com/personalize/latest/dg/limits.html)略低一些。

* 经过身份验证的用户
* 至少 50 个独立用户
* 至少 100 个独立项目
* 每个用户至少 24 次交互 

大多数情况下，这是很容易实现的，如果您在一个类别中排名较低，您通常可以通过在另一个类别中拥有更高排名来弥补。

一般来说，您的数据不会是适合 Personalize 的完美形式，需要进行一些修改才能正确结构化。此笔记本会指导您完成所有这些任务。

首先，我们将使用最新的 MovieLens 数据集，该数据集拥有超过 2500 万次交互和丰富的项目元数据集合。该数据集还有一个较小版本，可用于缩短训练时间，同时仍与完整数据集具有相同的功能。将 USE_FULL_MOVIELENS 设置为 True 以使用完整数据集。

In [ ]:
USE_FULL_MOVIELENS = False

首先，您将下载数据集并使用下面的代码将数据集解压缩到一个新文件夹中。

In [ ]:
data_dir = "poc_data"
!mkdir $data_dir

if not USE_FULL_MOVIELENS:
    !cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
    !cd $data_dir && unzip ml-latest-small.zip
    dataset_dir = data_dir + "/ml-latest-small/"
else:
    !cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-25m.zip
    !cd $data_dir && unzip ml-25m.zip
    dataset_dir = data_dir + "/ml-25m/"

看看您下载的数据文件。

In [ ]:
!ls $dataset_dir

除了有几个 CSV 文件和一个 README 外，目前很多文件未知。接下来，我们将输出 README 以了解更多信息！

In [ ]:
!pygmentize $dataset_dir/README.txt

在 README 中，我们看到有一个文件 `ratings.csv` 应该充当我们交互数据的代理，毕竟对一部电影进行评级肯定需要与之交互。该数据集还包含一些流派信息，作为电影流派组数据。在本 POC 中，我们将重点关注交互和流派数据。


## 准备数据<a class="anchor" id="prepare"></a>
[返回页首](#top)

接下来要做的是加载数据并确认数据处于良好状态，然后将数据保存到 CSV 中，以便用于 Amazon Personalize。

首先，导入数据科学中常用的 Python 库集合。

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

接下来，打开数据文件并查看前几行。

In [ ]:
original_data = pd.read_csv(dataset_dir + '/ratings.csv')
original_data.head(5)

In [ ]:
original_data.shape

In [ ]:
original_data.describe()

这表明我们有一个很好的 `userId` 和 `movieId` 值范围。接下来，确认数据格式始终是一个好主意。

In [ ]:
original_data.info()

In [ ]:
original_data.isnull().any()

通过此 POC，您可以看到数据集中总共有 25000095 个条目（对于完整数据集）/100836 个条目（对于较小版本数据集），分为 4 列，每个单元格存储为 int64 格式，但评级除外，它采用 float64 格式。

Int64 格式显然适用于 `userId` 和 `movieId`。但是，我们需要更深入地了解数据中的时间戳。要使用 Amazon Personalize，您需要以 [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time) 格式保存时间戳。

目前，时间戳值不是人类可读的。因此，让我们获取一个任意的时间戳值，并找出如何解释它。

通过选择任意时间戳并将时间戳转换为人类可读的格式，对转换后的数据集进行快速的可用性检查。

In [ ]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

这个日期作为时间戳是有意义的，因此我们可以继续格式化其余的数据。请记住，我们需要的数据是用户-项目-交互数据，在本例中是 `userId`、`movieId` 和 `timestamp`。我们的数据集另外还有一列 `rating`，在我们利用它来关注积极的交互后，可以从该数据集中删除该列。

由于这是一个显式反馈电影评级数据集，其中包括评级 1 到 5 的电影，我们希望仅包括用户"点赞"的电影，并模拟一个更像是 VOD 平台收集的数据的隐式数据集。为此，我们将筛选出评级低于 2（共 5 级）的所有交互，并创建一个"点击"EVENT_Type 和一个"观看"EVENT_Type。然后，我们将所有 2 级及以上的电影指定为"点击"，4 级及以上的电影指定为"点击"和"观看"。

请注意，这与我们正在建模的事件相对应：对于一个真实的数据集，您实际上会基于隐式反馈 [如点击、观看和/或显式反馈（如评级、点赞等）] 建模。

In [ ]:
watched_df = original_data.copy()
watched_df = watched_df[watched_df['rating'] > 3]
watched_df = watched_df[['userId', 'movieId', 'timestamp']]
watched_df['EVENT_TYPE']='watch'
watched_df.head()

In [ ]:
clicked_df = original_data.copy()
clicked_df = clicked_df[clicked_df['rating'] > 1]
clicked_df = clicked_df[['userId', 'movieId', 'timestamp']]
clicked_df['EVENT_TYPE']='click'
clicked_df.head()

In [ ]:
interactions_df = clicked_df.copy()
interactions_df = interactions_df.append(watched_df)
interactions_df.sort_values("timestamp", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 

In [ ]:
interactions_df.info()

让我们了解一下新数据集的功能 

In [ ]:
interactions_df.describe()

操作数据后，请务必确认数据格式是否已更改。

In [ ]:
interactions_df.dtypes

 Amazon Personalize 为用户、项目和时间戳提供了默认列名。这些默认列名为 `USER_ID`、`ITEM_ID` 和 `TIMESTAMP`。因此，对数据集的最后修改是用默认标题替换现有的列标题。

In [ ]:
interactions_df.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

就是这样！此时，数据已准备就绪，我们只需将数据保存为 CSV 文件。

In [ ]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## 创建数据集组和交互数据集<a class="anchor" id="group_dataset"></a>
[返回页首](#top)

Amazon Personalize 的最高隔离和抽象级别是*数据集组*。存储在其中一个数据集组中的信息不会影响任何其他数据集组或从其中一个数据集组创建的模型，这些数据集组是完全隔离的。这允许您运行许多实验，是我们如何保持您的模型的私密性并仅对您的数据进行全面训练的一部分。

在导入先前准备好的数据之前，需要有一个数据集组和一个添加到其中的数据集来处理交互。

数据集组可以包含以下类型的信息：

* 用户-项目-交互
* 事件流（实时交互）
* 用户元数据
* 项目元数据

在为交互数据创建数据集组和数据集之前，让我们验证您的环境是否可以成功地与 Amazon Personalize 通信。

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### 创建数据集组

下面的单元格将创建一个名为 `personalize-poc-lastfm` 的新数据集组。

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-poc-movielens"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

在使用数据集组之前，它必须处于活跃状态。这可能需要一两分钟。执行下面的单元格并等待单元格显示 ACTIVE 状态。它每秒检查一次数据集组的状态，工作最长可达 3 小时。

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

现在，您有了一个数据集组，您可以为交互数据创建一个数据集。

### 创建数据集

首先，定义一个架构，告诉 Amazon Personalize 您正在上载的数据集的类型。根据数据集的类型，架构中需要几个保留关键字和强制关键字。更详细的信息可以在[文档](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)中找到。

在这里，您将为需要 `USER_ID`、`ITEM_ID` 和 `TIMESTAMP` 字段的项目元数据创建架构。它们在架构中的定义顺序必须与它们在数据集中的出现顺序相同。

In [ ]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-poc-movielens-interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

创建架构后，可以在数据集组中创建数据集。请注意，这还不会加载数据。加载数据将在几个步骤后发生。

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-poc-movielens-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## 配置 S3 存储桶和 IAM 角色<a class="anchor" id="bucket_role"></a>
[返回页首](#top)

到目前为止，我们已下载、操作数据并将数据保存到运行此 Jupyter 笔记本的实例所连接的 Amazon EBS 实例中。但是，Amazon Personalize 需要一个 S3 存储桶充当您的数据源以及用于访问该存储桶的 IAM 角色。让我们把一切都安排好。

使用存储在此 Amazon SageMaker 笔记本底层的实例上的元数据来确定运行区域。如果您使用的是 Amazon SageMaker 以外的 Jupyter 笔记本，只需将区域定义为下面的字符串。Amazon S3 存储桶需要与我们目前创建的 Amazon Personalize 资源位于同一区域。

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

Amazon S3 存储桶名称是全局唯一的。要创建唯一的存储桶名称，下面的代码会将字符串 `personalizepocvod` 附加到您的 AWS 账号。然后，它会使用此名称在前一个单元格中发现的区域创建一个存储桶。

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizepocvod"
print(bucket_name)
if region == "us-east-1":
    s3.create_bucket(Bucket=bucket_name)
else:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
        )

### 将数据上载到 S3

现在，您的 Amazon S3 存储桶已经创建，请上载包含我们的用户-项目-交互数据的 CSV 文件。 

In [ ]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### 设置 S3 存储桶策略
Amazon Personalize 需要能够读取 S3 存储桶的内容。因此，添加可实现此操作的存储桶策略。

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

### 创建 IAM 角色

Amazon Personalize 需要能够承担 AWS 中的角色，以便拥有执行某些任务的权限。让我们来创建一个 IAM 角色并将所需的策略附加到该角色。下面的代码附加了非常宽松的策略；请对任何生产应用程序使用更严格的策略。

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRolePOC"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## 导入交互数据<a class="anchor" id="import"></a>
[返回页首](#top)

之前，您创建了数据集组和数据集来存放您的信息，因此现在您将执行一个导入作业，将数据从 S3 存储桶加载到 Amazon Personalize 数据集。 

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-import1",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

在使用数据集之前，导入作业必须处于活跃状态。执行下面的单元格并等待单元格显示 ACTIVE 状态。它每秒检查一次导入作业的状态，时间最长 6 小时。

导入数据可能需要一些时间，具体取决于数据集的大小。在此研讨会中，数据导入作业大约需要 15 分钟。

In [ ]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

当数据集导入处于活跃状态时，您可以开始使用 SIMS、Personalized-Ranking、Popularity-Count 和 User Personalization 来构建模型。此过程将在其他笔记本中继续。运行下面的单元格，然后继续存储一些值，以便在下一个笔记本中使用。

In [ ]:
%store USE_FULL_MOVIELENS
%store dataset_dir
%store interactions_dataset_arn
%store dataset_group_arn
%store bucket_name
%store role_arn
%store role_name
%store data_dir
%store region
%store interaction_schema_arn